# We want to use albumentations on public dataset and then indicate the ground truth and predictions

## Since we already have the functions to indicate where ground truth and predictions are we just need to make a function to use weather augmentations on the public dataset

There are 4 categories
- Rain
- SunFlare
- RandomShadom
- RandomFog

In [2]:
%pip install -U albumentations
import albumentations as A

  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.23
    Uninstalling albucore-0.0.23:
      Successfully uninstalled albucore-0.0.23
  Attempting uninstall: albumentations
    Found existing installation: albumentations 2.0.5
    Uninstalling albumentations-2.0.5:
      Successfully uninstalled albumentations-2.0.5
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\ewald\Documents\un-number-detection\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import os
import random
from PIL import Image
from typing import Union
import cv2
def generate_augmented_images(
    input_image_path: str,
    output_dir: str,
    augmentation: Union[str, A.Compose],
    num_augmentations: int = 5,
    seed: int = None,
    quality: int = 95,
    prefix: str = "aug",
    verbose: bool = True,
    augmentation_presets: dict = None
) -> list:
    """
    Generate and save augmented images to a specified directory.
    
    Parameters:
    - input_image_path: Path to source image (str)
    - output_dir: Output directory path (str) - will be created if not exists
    - augmentation: Albumentations transform or preset name ('rain', 'fog', etc.)
    - num_augmentations: Number of augmented versions to create (int)
    - seed: Optional random seed for reproducibility (int)
    - quality: Output JPEG quality (1-100)
    - prefix: Filename prefix for output images
    - verbose: Print progress messages
    
    Returns:
    List of saved file paths (list[str])
    """
    # Validate inputs
    if not os.path.isfile(input_image_path):
        raise FileNotFoundError(f"Input image not found: {input_image_path}")
    
    if seed is not None:
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Load image
    image = cv2.cvtColor(cv2.imread(input_image_path), cv2.COLOR_BGR2RGB)
    base_name = os.path.splitext(os.path.basename(input_image_path))[0]
    
    # Only initialize augmentation presets if not provided
    if augmentation_presets is None:
        # Define default augmentation presets
        augmentation_presets = {
            'rain': A.RandomRain(
                brightness_coefficient=0.9,  # Slightly darker rain
                drop_width=3, # Thicker rain drops
                blur_value=5, # Stronger blur effect
                p=1, # Always apply rain
                drop_length=20), # Longer rain streaks
            'sun_flare': A.RandomSunFlare(
                flare_roi=(0, 0, 1, 0.5), # Flare in the upper half of the image
                angle_lower=0.5, # Angle of tronger sun flare
                p=1), # Always apply sun flare
            'shadow': A.RandomShadow(
                num_shadows_lower=10, # More shadows
                num_shadows_upper=15, 
                shadow_dimension=8, # Larger and darker shadows
                shadow_roi=(0, 0, 1, 1), # Shadows across the entire image
                p=1), # Always apply shadows
            'fog': A.RandomFog(p=1), # Always apply fog
            
            # 'snow': A.RandomSnow(p=1) # Snow is really unrealistic and not recommended
        }

    
    # Configure augmentation pipeline
    if isinstance(augmentation, str):
        if augmentation not in augmentation_presets:
            raise ValueError(f"Unknown preset: {augmentation}. Available: {list(augmentation_presets.keys())}")
        transform = A.Compose([augmentation_presets[augmentation]])
    else:
        transform = augmentation
    
    # Generate augmented images
    saved_paths = []
    for i in range(num_augmentations):
        try:
            augmented = transform(image=image)['image']
            output_path = os.path.join(output_dir, f"{augmentation}-{i}-{prefix}_{base_name}.jpg")
            
            Image.fromarray(augmented).save(
                output_path,
                quality=quality,
                optimize=True,
                subsampling=0  # Keep highest chroma resolution
            )
            
            saved_paths.append(output_path)
            
            if verbose:
                print(f"Saved: {output_path}")
                
        except Exception as e:
            print(f"Error generating augmentation {i+1}: {str(e)}")
    
    return saved_paths

In [ ]:
input_path = "images/hazard_plate.jpg"
generate_augmented_images(
    input_image_path=input_path,
    output_dir="images/augmented",
    augmentation="fog",
    num_augmentations=1,
    seed=42  # Same seed = same augmentations
)

In [4]:
import os

# List of augmentations in the order you want them applied
augmentations = ["rain", "sun_flare", "shadow", "fog"]

# Input images directory
directory = "../../data/public_dataset/test/images"

# Where to store augmented outputs
directory_output = "output"

# Create subfolders (one for each augmentation)
for aug in augmentations:
    os.makedirs(os.path.join(directory_output, aug), exist_ok=True)

# Loop through each file in 'directory'
for filename in os.listdir(directory):
    # Make sure it's an image
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        input_path = os.path.join(directory, filename)
        
        # For each augmentation in our list
        for i, augmentation in enumerate(augmentations, start=1):
            # The folder where we’ll place augmented images for this augmentation
            output_dir = os.path.join(directory_output, augmentation)
            
            # If your generate_augmented_images function expects a directory
            # and internally names files "1.jpg", "2.jpg", etc., just call it:
            generate_augmented_images(
                input_image_path=input_path,
                output_dir=output_dir,
                augmentation=augmentation,
                num_augmentations=1,
                seed=42
            )

C:\Users\ewald\AppData\Local\Temp\ipykernel_14968\3626081996.py:72: UserWarning: Argument(s) 'angle_lower' are not valid for transform RandomSunFlare
  'sun_flare': A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5), angle_lower=0.5, p=1),
C:\Users\ewald\AppData\Local\Temp\ipykernel_14968\3626081996.py:73: UserWarning: Argument(s) 'num_shadows_lower, num_shadows_upper' are not valid for transform RandomShadow
  'shadow': A.RandomShadow(num_shadows_lower=10, num_shadows_upper=15, shadow_dimension=8, shadow_roi=(0, 0, 1, 1), p=1),


Saved: output\rain\rain-0-aug_10.jpg
Saved: output\sun_flare\sun_flare-0-aug_10.jpg
Saved: output\shadow\shadow-0-aug_10.jpg
Saved: output\fog\fog-0-aug_10.jpg
Saved: output\rain\rain-0-aug_100.jpg
Saved: output\sun_flare\sun_flare-0-aug_100.jpg
Saved: output\shadow\shadow-0-aug_100.jpg
Saved: output\fog\fog-0-aug_100.jpg
Saved: output\rain\rain-0-aug_101.jpg
Saved: output\sun_flare\sun_flare-0-aug_101.jpg
Saved: output\shadow\shadow-0-aug_101.jpg
Saved: output\fog\fog-0-aug_101.jpg
Saved: output\rain\rain-0-aug_102.jpg
Saved: output\sun_flare\sun_flare-0-aug_102.jpg
Saved: output\shadow\shadow-0-aug_102.jpg


KeyboardInterrupt: 